**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
FOLDER="annotation_enhancer_zscore_junke"

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER)
for(fname in dir(fdiry)){print(fname)}

[1] "description.tsv"
[1] "enhancer_zscore_junke.inactive.ASTARR.bed.gz"
[1] "enhancer_zscore_junke.inactive.ASTARR.merge.bed.gz"
[1] "enhancer_zscore_junke.inactive.LMPRA.bed.gz"
[1] "enhancer_zscore_junke.inactive.LMPRA.merge.bed.gz"
[1] "enhancer_zscore_junke.inactive.TMPRA_OL13.bed.gz"
[1] "enhancer_zscore_junke.inactive.TMPRA_OL43.bed.gz"
[1] "enhancer_zscore_junke.inactive.TMPRA_OL45.bed.gz"
[1] "enhancer_zscore_junke.inactive.TMPRA.bed.gz"
[1] "enhancer_zscore_junke.inactive.TMPRA.merge.bed.gz"
[1] "enhancer_zscore_junke.inactive.WSTARR.bed.gz"
[1] "enhancer_zscore_junke.inactive.WSTARR.merge.bed.gz"
[1] "enhancer_zscore_junke.peak.ASTARR_A.bed.gz"
[1] "enhancer_zscore_junke.peak.ASTARR_AB.bed.gz"
[1] "enhancer_zscore_junke.peak.ASTARR_R.bed.gz"
[1] "enhancer_zscore_junke.peak.ASTARR.concat.bed.gz"
[1] "enhancer_zscore_junke.peak.ASTARR.merge.bed.gz"
[1] "enhancer_zscore_junke.peak.LMPRA_A.bed.gz"
[1] "enhancer_zscore_junke.peak.LMPRA_AB.bed.gz"
[1] "enhancer_zscore_junke.peak.L

In [4]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)

fnames = dir(fdiry)
fnames = grep("peak.annotation", fnames, value = TRUE)
fnames = grep("merge", fnames, value = TRUE, invert = TRUE)
for(fname in fnames){print(fname)}

[1] "peak.annotation.enhancer_zscore_junke.peak.ASTARR_A.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.ASTARR_AB.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.ASTARR_R.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.ASTARR.concat.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.LMPRA_A.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.LMPRA_AB.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.LMPRA_R.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.LMPRA.concat.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.TMPRA_A.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.TMPRA_R.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.TMPRA.concat.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.WSTARR_A.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.WSTARR_AB.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.WSTARR_R.bed.gz"
[1] "peak.annotation.enhancer_zscore_junke.peak.WSTARR.concat.bed.gz"


## Import data

In [5]:
fdiry  = file.path(FD_RES, "results", "region", FOLDER)
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_cnames = dat
print(dim(dat))
dat

[1] 9 2


Name,Description
<chr>,<chr>
Chrom,Seqnames
Start,Start
End,End
Name,Name
Score,Z_score based on mean logFC
Strand,Strand
Assay_Type,Assay type
Assay_Label,Assay label
Dataset,Dataset


In [6]:
fnames = dir(fdiry)
fnames = grep("peak.annotation", fnames, value = TRUE)
fnames = grep("merge", fnames, value = TRUE, invert = TRUE)
fnames %>% 
    str_split(., "\\.") %>%
    lapply(., FUN = `[`, 5)

list()

In [9]:
### init: set column names
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)

fnames = dir(fdiry)
fnames = grep("peak.annotation", fnames, value = TRUE)
fnames = grep("merge", fnames, value = TRUE, invert = TRUE)
fpaths = file.path(fdiry, fnames)
#fname  = paste("peak.annotation", "peak", "bed.gz", sep="*")
#fglob  = file.path(fdiry, fname)
#fpaths = Sys.glob(fglob)
#fnames = basename(fpaths)

### init: annotation and label
annotation = "Enhancer_ZScore"
labels = fnames %>% 
    str_split(., "\\.") %>%
    lapply(., FUN = `[`, 5)
#labels     = fnames %>% 
#    str_split(., "\\.") %>%
#    do.call(rbind, .) %>%
#    as.data.frame %>%
#    dplyr::pull(5)

names(fnames) = labels
names(labels) = labels
print(fnames)

                                                         ASTARR_A 
     "peak.annotation.enhancer_zscore_junke.peak.ASTARR_A.bed.gz" 
                                                        ASTARR_AB 
    "peak.annotation.enhancer_zscore_junke.peak.ASTARR_AB.bed.gz" 
                                                         ASTARR_R 
     "peak.annotation.enhancer_zscore_junke.peak.ASTARR_R.bed.gz" 
                                                           ASTARR 
"peak.annotation.enhancer_zscore_junke.peak.ASTARR.concat.bed.gz" 
                                                          LMPRA_A 
      "peak.annotation.enhancer_zscore_junke.peak.LMPRA_A.bed.gz" 
                                                         LMPRA_AB 
     "peak.annotation.enhancer_zscore_junke.peak.LMPRA_AB.bed.gz" 
                                                          LMPRA_R 
      "peak.annotation.enhancer_zscore_junke.peak.LMPRA_R.bed.gz" 
                                                            LM

In [11]:
### Import data
lst = lapply(labels, function(label){
    ### set file path
    fname = fnames[label]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Peak_ATAC  = paste0(Chrom_ATAC, ":", Start_ATAC, "-", End_ATAC),
        Annotation = annotation,
        Label      = label,
        Region     = paste0(Chrom, ":", Start, "-", End),
        Value      = Score,
        Note       = Assay_Label
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 15
 [1] "ASTARR_A"  "ASTARR_AB" "ASTARR_R"  "ASTARR"    "LMPRA_A"   "LMPRA_AB" 
 [7] "LMPRA_R"   "LMPRA"     "TMPRA_A"   "TMPRA_R"   "TMPRA"     "WSTARR_A" 
[13] "WSTARR_AB" "WSTARR_R"  "WSTARR"   


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,Assay_Type,Assay_Label,Dataset,Overlap,Peak_ATAC,Annotation,Label,Region,Value,Note
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
chr1,777949,779437,chr1,778720,778830,peak1,2.009577,.,ASTARR,ASTARR_A,ASTARR_TR,110,chr1:777949-779437,Enhancer_ZScore,ASTARR_A,chr1:778720-778830,2.009577,ASTARR_A
chr1,1012902,1014502,chr1,1013010,1013570,peak2,2.155097,.,ASTARR,ASTARR_A,ASTARR_TR,560,chr1:1012902-1014502,Enhancer_ZScore,ASTARR_A,chr1:1013010-1013570,2.155097,ASTARR_A
chr1,1063757,1064554,chr1,1064280,1064450,peak3,2.201032,.,ASTARR,ASTARR_A,ASTARR_TR,170,chr1:1063757-1064554,Enhancer_ZScore,ASTARR_A,chr1:1064280-1064450,2.201032,ASTARR_A
chr1,1200699,1201823,chr1,1200920,1201050,peak5,2.375591,.,ASTARR,ASTARR_A,ASTARR_TR,130,chr1:1200699-1201823,Enhancer_ZScore,ASTARR_A,chr1:1200920-1201050,2.375591,ASTARR_A
chr1,1304789,1306435,chr1,1305780,1305880,peak6,1.990023,.,ASTARR,ASTARR_A,ASTARR_TR,100,chr1:1304789-1306435,Enhancer_ZScore,ASTARR_A,chr1:1305780-1305880,1.990023,ASTARR_A
chr1,1344582,1346157,chr1,1344940,1345110,peak7,2.086690,.,ASTARR,ASTARR_A,ASTARR_TR,170,chr1:1344582-1346157,Enhancer_ZScore,ASTARR_A,chr1:1344940-1345110,2.086690,ASTARR_A


## Arrange and summarize

In [12]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### summarize
    dat = dat %>%
        dplyr::group_by(Chrom_ATAC, Start_ATAC, End_ATAC, Peak_ATAC, Annotation, Label) %>%
        dplyr::summarise(
            Count  = n(), 
            Region = paste(Region, collapse="|"),
            Score  = mean(Value),
            Note   = paste(sort(unique(Note)), collapse="|"),
            .groups = "drop")
    
    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_ATAC, Start_ATAC, End_ATAC) %>%
        dplyr::select(-Chrom_ATAC, -Start_ATAC, -End_ATAC) %>%
        dplyr::rename("Peak" = Peak_ATAC)
    return(dat)
})

### assign and show
lst_peak_annot_summary = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 15
 [1] "ASTARR_A"  "ASTARR_AB" "ASTARR_R"  "ASTARR"    "LMPRA_A"   "LMPRA_AB" 
 [7] "LMPRA_R"   "LMPRA"     "TMPRA_A"   "TMPRA_R"   "TMPRA"     "WSTARR_A" 
[13] "WSTARR_AB" "WSTARR_R"  "WSTARR"   


Peak,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:777949-779437,Enhancer_ZScore,ASTARR_A,1,chr1:778720-778830,2.009577,ASTARR_A
chr1:1012902-1014502,Enhancer_ZScore,ASTARR_A,1,chr1:1013010-1013570,2.155097,ASTARR_A
chr1:1063757-1064554,Enhancer_ZScore,ASTARR_A,1,chr1:1064280-1064450,2.201032,ASTARR_A
chr1:1200699-1201823,Enhancer_ZScore,ASTARR_A,1,chr1:1200920-1201050,2.375591,ASTARR_A
chr1:1304789-1306435,Enhancer_ZScore,ASTARR_A,1,chr1:1305780-1305880,1.990023,ASTARR_A
chr1:1344582-1346157,Enhancer_ZScore,ASTARR_A,1,chr1:1344940-1345110,2.086690,ASTARR_A


## Explore and check results

In [13]:
lst = lst_peak_annot_summary
lst = lst[c("ASTARR_AB", "ASTARR_A", "ASTARR_R", "ASTARR")]
lst = lapply(lst, function(dat){
    res = table(dat$Note)
    return(res)
})
print(lst)

$ASTARR_AB

ASTARR_AB 
     2799 

$ASTARR_A

ASTARR_A 
    7427 

$ASTARR_R

ASTARR_R 
   11371 

$ASTARR

                   ASTARR_A          ASTARR_A|ASTARR_AB 
                       3680                        2593 
ASTARR_A|ASTARR_AB|ASTARR_R           ASTARR_A|ASTARR_R 
                        181                         973 
                  ASTARR_AB          ASTARR_AB|ASTARR_R 
                         19                           6 
                   ASTARR_R 
                      10211 



In [14]:
lst = lst_peak_annot_summary
lst = lst[c("ASTARR_AB", "ASTARR_A", "ASTARR_R", "ASTARR")]
lst = lapply(lst, function(dat){
    res =  table(dat$Count)
    return(res)
})
print(lst)

$ASTARR_AB

   1    2    3 
2746   51    2 

$ASTARR_A

   1    2    3    4    5 
6778  599   44    4    2 

$ASTARR_R

   1    2    3    4    5    6    7   10 
7669 2618  797  214   53   13    6    1 

$ASTARR

    1     2     3     4     5     6     7     8     9    11 
10351  5523  1248   389   102    33    11     3     2     1 



## Save results

In [15]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "summary")

lst = lst_peak_annot_summary
for (idn in names(lst)){
    fname = paste("peak.summary.enhancer_zscore_junke", idn,"tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()
    
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.summary.enhancer_zscore_junke.ASTARR_A.tsv"
[1] "peak.summary.enhancer_zscore_junke.ASTARR_AB.tsv"
[1] "peak.summary.enhancer_zscore_junke.ASTARR_R.tsv"
[1] "peak.summary.enhancer_zscore_junke.ASTARR.tsv"
[1] "peak.summary.enhancer_zscore_junke.LMPRA_A.tsv"
[1] "peak.summary.enhancer_zscore_junke.LMPRA_AB.tsv"
[1] "peak.summary.enhancer_zscore_junke.LMPRA_R.tsv"
[1] "peak.summary.enhancer_zscore_junke.LMPRA.tsv"
[1] "peak.summary.enhancer_zscore_junke.TMPRA_A.tsv"
[1] "peak.summary.enhancer_zscore_junke.TMPRA_R.tsv"
[1] "peak.summary.enhancer_zscore_junke.TMPRA.tsv"
[1] "peak.summary.enhancer_zscore_junke.WSTARR_A.tsv"
[1] "peak.summary.enhancer_zscore_junke.WSTARR_AB.tsv"
[1] "peak.summary.enhancer_zscore_junke.WSTARR_R.tsv"
[1] "peak.summary.enhancer_zscore_junke.WSTARR.tsv"
